# Import librairie #

In [12]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

## Charger les variables d'environnement ##

In [13]:
# Charger le .env depuis la racine du projet
from pathlib import Path
dotenv_path = r'C:\Penny\.env'
load_result = load_dotenv(dotenv_path=dotenv_path)
print(f"Chargement .env : {'[OK] Reussi' if load_result else '[ERREUR] Echec'}")

Chargement .env : ✓ Réussi


## Connexion Redshift Pennylane ##

In [14]:
conn = psycopg2.connect(
    host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
    port=5439,
    dbname='prod',
    user='u_289572',
    password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
)

print("Connexion Redshift etablie")

Connexion Redshift etablie


## Fonction helper : Connexion auto-reconnect ##

In [15]:
def get_active_connection():
    """Retourne une connexion active, la recrée si nécessaire"""
    global conn
    try:
        # Test si la connexion est active
        pd.read_sql("SELECT 1;", conn)
        return conn
    except:
        # Reconnexion si fermée
        print("⟳ Reconnexion Redshift...")
        conn = psycopg2.connect(
            host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
            port=5439,
            dbname='prod',
            user='u_289572',
            password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
        )
        print("✓ Reconnecté")
        return conn

# Utilisation : remplace "conn" par "get_active_connection()" dans tes requêtes
# Exemple :
df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())
print(f"Test réussi : {len(df)} ligne(s)")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_52208\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_52208\3220976796.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())


Test réussi : 1 ligne(s)


## Liste table Balance Pennylane ##

In [16]:
# Tables trial balance Pennylane
tables_extended = [
    'trial_balance'
]

accessible_tables = []

for table in tables_extended:
    try:
        query = f"SELECT * FROM pennylane.{table} LIMIT 1;"
        df = pd.read_sql(query, conn)
        accessible_tables.append({
            'table_name': table,
            'columns': len(df.columns),
            'column_list': ', '.join(df.columns.tolist()[:5]) + '...'
        })
        print(f"✓ {table} ({len(df.columns)} colonnes)")
    except:
        pass

print(f"\n\n=== RÉSUMÉ: {len(accessible_tables)} tables accessibles ===")
df_accessible = pd.DataFrame(accessible_tables)
print(df_accessible.to_string(index=False))

C:\Users\ycloa\AppData\Local\Temp\ipykernel_52208\4180995964.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✓ trial_balance (16 colonnes)


=== RÉSUMÉ: 1 tables accessibles ===
   table_name  columns                                       column_list
trial_balance       16 company_id, company_name, id, number, vat_rate...


## Liste colonnes Balance Pennylane ##

In [17]:
# Afficher les colonnes trial balance
tables_found = [
    'trial_balance'
]

for table in tables_found:
    query = f"SELECT * FROM pennylane.{table} LIMIT 0;"  # LIMIT 0 = structure seulement
    df = pd.read_sql(query, conn)
    print(f"\n{'='*60}")
    print(f"Table: pennylane.{table}")
    print(f"{'='*60}")
    print(f"Colonnes ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")


Table: pennylane.trial_balance
Colonnes (16):
   1. company_id
   2. company_name
   3. id
   4. number
   5. vat_rate
   6. country_alpha2
   7. label
   8. enabled
   9. debits
  10. credits
  11. balance
  12. ledger_events_count
  13. notes_count
  14. fiscal_period_start
  15. fiscal_period_finish
  16. updated_at


C:\Users\ycloa\AppData\Local\Temp\ipykernel_52208\1042180433.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## Chargement des données balance ##

In [18]:
# Charger les balances
query_gl = "SELECT * FROM pennylane.trial_balance;"
df_trial_balance = pd.read_sql(query_gl, get_active_connection())

print(f"✓ Données chargées : {len(df_trial_balance)} lignes, {len(df_trial_balance.columns)} colonnes")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_52208\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_52208\3270579235.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_trial_balance = pd.read_sql(query_gl, get_active_connection())


✓ Données chargées : 163 lignes, 16 colonnes


## Affichage des colonnes disponibles ##

In [19]:
print("Colonnes disponibles :")
for i, col in enumerate(df_trial_balance, 1):
    print(f"  {i:2d}. {col}")

Colonnes disponibles :
   1. company_id
   2. company_name
   3. id
   4. number
   5. vat_rate
   6. country_alpha2
   7. label
   8. enabled
   9. debits
  10. credits
  11. balance
  12. ledger_events_count
  13. notes_count
  14. fiscal_period_start
  15. fiscal_period_finish
  16. updated_at


## Affichage type colonne ##

In [20]:
print(f"Types de données des {len(df_trial_balance.columns)} colonnes :\n")
for i, (col, dtype) in enumerate(df_trial_balance.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

Types de données des 16 colonnes :

   1. company_id                     → int64
   2. company_name                   → object
   3. id                             → float64
   4. number                         → object
   5. vat_rate                       → object
   6. country_alpha2                 → object
   7. label                          → object
   8. enabled                        → bool
   9. debits                         → float64
  10. credits                        → float64
  11. balance                        → float64
  12. ledger_events_count            → int64
  13. notes_count                    → object
  14. fiscal_period_start            → object
  15. fiscal_period_finish           → object
  16. updated_at                     → datetime64[ns]


## Conversion des types de colonnes ##

In [21]:
# Conversion des types
df_trial_balance = df_trial_balance.astype({
    'fiscal_period_start':'datetime64[ns]',
    'debits':'float64',
    'credits':'float64',
    'balance':'float64',
    'fiscal_period_finish':'datetime64[ns]',
    'updated_at':'datetime64[ns]'
})

# Convertir toutes les autres colonnes en string
columns_to_string = [col for col in df_trial_balance.columns if col not in ['fiscal_period_start','debits','credits','balance','fiscal_period_finish','updated_at']]
df_trial_balance[columns_to_string] = df_trial_balance[columns_to_string].astype(str)

print("✓ Conversion effectuée\n")
print("Nouveaux types :")
for i, (col, dtype) in enumerate(df_trial_balance.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

✓ Conversion effectuée

Nouveaux types :
   1. company_id                     → object
   2. company_name                   → object
   3. id                             → object
   4. number                         → object
   5. vat_rate                       → object
   6. country_alpha2                 → object
   7. label                          → object
   8. enabled                        → object
   9. debits                         → float64
  10. credits                        → float64
  11. balance                        → float64
  12. ledger_events_count            → object
  13. notes_count                    → object
  14. fiscal_period_start            → datetime64[ns]
  15. fiscal_period_finish           → datetime64[ns]
  16. updated_at                     → datetime64[ns]


## Connexion PostgreSQL local ##

In [22]:
# Connexion à PostgreSQL local (utilise les variables du .env)
conn_pg = psycopg2.connect(
    host=os.getenv('POSTGRES_HOST'),
    port=int(os.getenv('POSTGRES_PORT')),
    dbname=os.getenv('POSTGRES_DB'),
    user=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD')
)

print("✓ Connexion PostgreSQL établie")
print(f"  Database: {os.getenv('POSTGRES_DB')}")
print(f"  User: {os.getenv('POSTGRES_USER')}")
print(f"  Port: {os.getenv('POSTGRES_PORT')}")

✓ Connexion PostgreSQL établie
  Database: pennylane_data
  User: pennylane_user
  Port: 5433


## Import des données dans PostgreSQL avec SQLAlchemy ##

In [23]:
from sqlalchemy import create_engine

# Créer un engine SQLAlchemy
engine = create_engine(
    f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DB')}"
)

print("✓ Engine SQLAlchemy créé")

✓ Engine SQLAlchemy créé


## Export du DataFrame vers PostgreSQL ##

## Export vers PostgreSQL local ##

In [ ]:
# Export DataFrame vers PostgreSQL local
df_trial_balance.to_sql(
    name='trial_balance',
    con=engine,
    schema='pennylane',
    if_exists='replace',
    index=False,
    method='multi',
    chunksize=1000
)

print(f"[OK] Table 'trial_balance' exportee : {len(df_trial_balance)} lignes")

# Fermer connexions
conn.close()
conn_pg.close()

print("[OK] Connexions fermees")